In [1]:
!pip install datasets transformers

In [16]:
# lr for BERT 0.002
# lr for decoder 0.2

In [17]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
#datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'valid.txt'})

Using the latest cached version of the module from C:\Users\ewako\.cache\huggingface\modules\datasets_modules\datasets\wikitext\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Thu Jan 20 23:18:06 2022) since it couldn't be found locally at wikitext., or remotely on the Hugging Face Hub.
Reusing dataset wikitext (C:\Users\ewako\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
import math
from transformers import AutoTokenizer

In [19]:
model_checkpoint = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [15]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

NameError: name 'tokenizer' is not defined

In [22]:
import torch.nn.functional as F
import torch
def getNovelMask(target, vocab_size):
    b, l = target.size()
    zeros = torch.zeros(b, l, vocab_size).to(target.device)
    ones = torch.ones(b, l, vocab_size).to(target.device)

    target_index = target.unsqueeze(1).expand(b, l, l).transpose(-2, -1).triu().transpose(-2, -1)
    matrix = zeros.scatter_add_(2, target_index, ones)
    matrix[:, :, 0] = 0
    summ_true = torch.tensor(range(1, l + 1)).unsqueeze(0).float().to(target.device)
    summ_now = torch.sum(matrix, dim=-1)
    diff = summ_true - summ_now
    matrix[:, :, 0] = diff
    matrix = torch.cat((torch.zeros(b, 1, vocab_size).to(target.device), matrix[:, :-1, :]), 1)
    novel_mask = matrix < 1.

    return novel_mask


def sg_loss(model, batch, gamma, output):
    longer_sample = batch['input_ids'][0]
    inp = longer_sample
    model_output = output
    target = batch['labels'][0]
    logits = model_output[1]

    # ScaleGrad
    probs = F.softmax(logits, dim=-1)
    novel_mask = getNovelMask(target.unsqueeze(0), logits.size(-1))
    rep_mask = ~novel_mask

    new_probs = probs * novel_mask * gamma + probs * rep_mask + 1e-8
    new_probs = F.normalize(new_probs, p=1, dim=-1)
    lprobs = torch.log(new_probs)
    lprobs_flatten = torch.unsqueeze(lprobs[0,:,:],0)
    assert lprobs_flatten.size(0) == 1, 'Nonflat sequence error'
    loss = F.nll_loss(lprobs_flatten[0], target, reduction='sum')
    true_token_logits = -F.nll_loss(logits[0], target, reduction='none')
    ntokens = inp.numel()

    return loss / ntokens

In [23]:
gamma = 0.2
class ScaleGradTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = sg_loss(model, inputs, gamma, outputs)
        return (loss, outputs) if return_outputs else loss

In [24]:
from transformers import Trainer, TrainingArguments

In [25]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs = 1.0
)

In [26]:
import numpy as np
from datasets import load_metric

# source: https://huggingface.co/docs/transformers/training

def compute_metrics(eval_pred):
    # TODO: zmienić
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
trainer = ScaleGradTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,    # compute_metrics: Optional[Callable[[EvalPrediction], Dict]] = None, 
    # loss jest podstawowym założeniem (było, że jeśli tutaj None, to only_loss na true)
    train_dataset=None,
    eval_dataset=None
)

NameError: name 'model' is not defined

In [28]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

NameError: name 'trainer' is not defined

In [ ]:
# TODO: na inne miary?
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [29]:
from datasets import list_metrics
metrics_list = list_metrics()

In [30]:
print(metrics_list) # nie widzę Rep/l and uniq

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'chrf', 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'google_bleu', 'indic_glue', 'matthews_correlation', 'mauve', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'wer', 'wiki_split', 'xnli']


In [ ]:
# tu są metryki z torcha: https://torchmetrics.readthedocs.io/en/latest/references/modules.html#text